In [ ]:
%%capture
!pip install pypdf2 textract

In [62]:
import numpy as np
import pandas as pd
import re
import textract

from google.colab import drive
from PyPDF2 import PdfReader

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
def get_title_cleanup() -> dict:
  return {
    # Bad title: Good Title
  }

In [4]:
def get_department_cleanup(year:int) -> dict:
  results = {
    # Bad department: good department
  }

  return results

In [5]:
def get_start_index(year:int, team:str, page:int) -> int:
  if year == 2019 and team == 'DET' and page == 6:
    return 8
  else:
    return 0

In [6]:
def cleanup_name_title(year:int, team:str, title:str, original_name:str):
  new_title = title
  new_name = str(original_name)

  if new_title in [
      'Bill “Chief ” Gayton, Bradley Kelley, Pat Murtaugh, Mike Piatnik,'
  ]:
    new_name = new_title
    

  # Remove banned substrings
  banned_substrings = [
      'Dr.', 'O.D.', 'M.D.'
  ]

  for e in banned_substrings:
    new_name = new_name.replace(e, '')


  # Trim unwanted spaces and characters
  new_name = new_name.strip(' ').strip(',').replace("' ", "'")

  return new_title, new_name

In [7]:
def get_front_office_roster(pdf:PdfReader, year:int, team:str, first_page:int, last_page:int) -> pd.DataFrame:

  def get_department(row):
    if row['Name'] == 'None':
      return row['Title']
    else:
      return None


  df_main = pd.DataFrame()
  pages = range(first_page + 1, last_page + 2)

  for p in pages:
    pdf_page = pdf.getPage(p)
    start = get_start_index(year, team, p)

    records = [record for record in pdf_page.extractText().split('\n')][start:]
    return records
    # df = pd.DataFrame(records)

    # title_name_split = [re.split(' ?\.\.+ ?', record) for record in records]
    # df = pd.DataFrame([e[:2] + [year, team] for e in title_name_split],
    #                   columns=['Title', 'Name', 'Year', 'Team'])


    # Cleanup job titles
    # df['Title'] = df['Title'].str.strip(' ')
    # df['Title'] = df['Title'].replace(get_title_cleanup())
    # df['Prior Title'] = df.Title.shift(1)


    # 


    # # # # Expand name tuples to multiple rows, if necessary
    # df = df.explode('Name', ignore_index=True)


    # # # Determine the department
    # df['Department'] = df.apply(lambda x: get_department(x), axis=1) \
    #                      .replace(get_department_cleanup(year))
    # df['Department'] = df['Department'].fillna(method='ffill')


    # # # Remove empty rows
    # df = df[df.Name != 'None']


    # # # Order the columns as specified by SABR BoB Committee,
    # df = df[['Year', 'Team', 'Name', 'Title', 'Department']]


    # Update the main dataframe
    df_main = pd.concat([df_main, df]).reset_index(drop=True)

  return df_main

In [ ]:
pages = {
  'ARI': {
    2014: [1, 2]
    # 2020: [4, 5],
    # 2021: [4, 5],
    # 2022: [4, 5]
  }
}

final_df = pd.DataFrame()

for team, contents in pages.items():
  for year, pdf_pages in contents.items():
    first_page = pdf_pages[0]
    last_page = pdf_pages[1]
    temp_df = get_front_office_roster(
                PdfReader(f'drive/My Drive/sabr/business-of-baseball/team-media-guides/{team}/{year}_{team}_Media_Guide.pdf'),
                year, team, first_page, last_page
              )
    break
    
    # final_df = pd.concat([final_df, temp_df])

In [ ]:
final_df = tabula.read_pdf(f'drive/My Drive/sabr/business-of-baseball/team-media-guides/ARI/2014_ARI_Media_Guide.pdf', pages=[1,2])

NameError: ignored

In [ ]:
pdf = PdfReader(f'drive/My Drive/sabr/business-of-baseball/team-media-guides/ARI/2014_ARI_Media_Guide.pdf')

In [ ]:
final_df.to_csv('drive/My Drive/sabr/business-of-baseball/team-media-guides/data/team-stage/det-front-office-rosters.csv', index=False)

In [ ]:
department_leadership = {
    2014: {
        # Leadership position: Department
        'Managing General Partner': 'Executive Office',
        'Vice President, Finance': 'Accounting',
        'Executive Assistant to the President & CEO': 'Administration/Consultants',
        'Vice President, Special Asst. to the GM': 'Baseball Operations',
        'Vice President, Broadcasting': 'Broadcasting',
        'Sr. Director, Game Ops & Multi-Media Productions': 'Game Ops & Multi-Media Productions',
        'Director, Player & Media Relations': 'Communications',
        'Vice President, Corporate & Community Impact': 'Community Affairs',
        'Vice President, Corporate Partnerships': 'Corporate Partnership',
        'Vice President, Facilities & Event Services': 'Facilities & Event Services',
        'Benefits Manager': 'Human Resources',
        'Vice President & Chief Information Officer': 'Information Technology',
        'Sr. Director, Legal Affairs & Associate General Counsel': 'Legal',
        'Vice President, Special Projects & Fan Experience': 'Special Projects',
        'Vice President, Marketing': 'Ticketing & Marketing'
    }
}

In [211]:
pages = {
  'ARI': {
    2014: [166, 563]
    # 2020: [4, 5],
    # 2021: [4, 5],
    # 2022: [4, 5]
  }
}

final_df = pd.DataFrame()

for team, contents in pages.items():
  for year, bookend in contents.items():
    pdf = textract.process(f'drive/My Drive/sabr/business-of-baseball/team-media-guides/{team}/{year}_{team}_Media_Guide.pdf', method='pdfminer')

    front_office = pdf.decode('utf-8').splitlines()[bookend[0]:bookend[1] + 1]
    records = [[y.strip(' ').strip('.') for y in x.split('..') if len(y.strip()) > 1] for x in front_office if x]
    df = pd.DataFrame(records).dropna(thresh=1)
    df[0] = df[0].apply(lambda x: re.sub(' +', ' ', x))
    df['Department'] = df.apply(lambda x: x[0] if x[0].isupper() else None, axis = 1)
    df['Department'] = df[0].map(department_leadership[year]).ffill()
    df = df[(~df['Department'].isna()) & ~(df[0].str.isupper())]

    # Cleanup job titles and names
    df['Title'] = df[0].str.strip(' ')
    df['Title'] = df['Title'].replace(get_title_cleanup())
    df['Prior Title'] = df.Title.shift(1)
    df['Name'] = df[[1, 'Title']].bfill(axis=1).iloc[:, 0]
    df[['Title', 'Name', 'Department']]
    df['Title'] = df.apply(lambda x: re.sub(str(x['Name']), '', x['Title']), axis=1)
    df['Title'].replace('', None, regex=False, inplace=True)
    df['Name'] = df['Name'].str.split(',')
    df = df.explode('Name')
    df = df[df['Name'] != '']

    df['Year'] = year
    df['Team'] = team

    df = df[['Year', 'Team', 'Name', 'Title', 'Department']]

    final_df = pd.concat([final_df, df]).reset_index(drop=True)

In [212]:
final_df

,Year,Team,Name,Title,Department
2,2014,ARI,Ken Kendrick,Managing General Partner,Executive Office
3,2014,ARI,Mike Chipman,General Partners,Executive Office
3,2014,ARI,Jeff Royer,General Partners,Executive Office
4,2014,ARI,Derrick Hall,President & Chief Executive Officer,Executive Office
5,2014,ARI,Tom Harris,Executive Vice President & Chief Financial Off...,Executive Office
...,...,...,...,...,...
340,2014,ARI,Diana Rodriguez,Administrative Assistant,Human Resources
342,2014,ARI,Bob Zweig,Vice President & Chief Information Officer,Information Technology
343,2014,ARI,Craig Pozen,"Sr. Director, Business Systems & Applications",Information Technology
344,2014,ARI,Jay Gaskin,"Director, IT Systems & Infrastructure",Information Technology


In [207]:
df[['Name', 'Title', 'Department']]

,Name,Title,Department
2,Ken Kendrick,Managing General Partner,Executive Office
3,Mike Chipman,General Partners,Executive Office
3,Jeff Royer,General Partners,Executive Office
4,Derrick Hall,President & Chief Executive Officer,Executive Office
5,Tom Harris,Executive Vice President & Chief Financial Off...,Executive Office
...,...,...,...
340,Diana Rodriguez,Administrative Assistant,Human Resources
342,Bob Zweig,Vice President & Chief Information Officer,Information Technology
343,Craig Pozen,"Sr. Director, Business Systems & Applications",Information Technology
344,Jay Gaskin,"Director, IT Systems & Infrastructure",Information Technology


# Explore new media guide

In [225]:
pdf = textract.process(f'drive/My Drive/sabr/business-of-baseball/team-media-guides/ARI/2015_ARI_Media_Guide.pdf')

In [226]:
front_office = pdf.decode('ascii').splitlines()[467:840 + 1]
records = [[y.strip(' ').strip('.') for y in x.split('AA') if len(y.strip()) > 1] for x in front_office if x]
df = pd.DataFrame(records).dropna(thresh=1)

UnicodeDecodeError: ignored

In [236]:
pdf.isascii()

False

In [244]:
front_office = re.sub('AA+', '..........', pdf.decode('utf-8')).splitlines()[467:840 + 1]
records = [[y.strip(' ').strip('.') for y in x.split('..') if len(y.strip()) > 1] for x in front_office if x]

In [245]:
records

[['>$%$.?%.&P/%/0$#&<$07%/0', 'Ken Kendrick'],
 ['P/%/0$#&<$07%/03', 'Mike Chipman, Jeff Royer'],
 ['President & Chief Executive Officer', 'Derrick Hall'],
 ['Executive Vice President & Chief Financial Officer', 'C"2&Q$00?3'],
 ['Chief Baseball Officer', 'C"%(&D$&\'F33$'],
 ['Executive Vice President, Business Operations', '1F##/%&>$I/('],
 ['Sr. Vice President, Ticket Sales & Marketing', 'John'],
 ['John'],
 ['Fisher'],
 [],
 ['Sr. Vice President & General Counsel', '!"%$&D//'],
 ['Sr. Vice President, Communications', 'Josh Rawitch'],
 ['Sr. Vice President, Chief HR & Diversity Officer', 'Marian Rhodes'],
 ['Sr. Vice President & General Manager', 'Dave Stewart'],
 ['Sr. Vice President, Baseball Operations', 'De Jon Watson Special'],
 ['Assistant to the President & CEO', 'DF?3&P"%@$#/@'],
 ['Special Assistant to the President & CEO', '\'"#$%=&Q/2"%='],
 ['Special Assistant to the President & CEO', 'Randy Johnson'],
 ['Special Assistant to the President & CEO', 'J.J. Putz'],
 ['ACCOUNTI